# Creating FITS output for any given SME output

## Author(s): Sven Buder (SB, WG4)

### History:
181011 SB Created

In [1]:
# Preamble for notebook 

# Compatibility with Python 3
from __future__ import (absolute_import, division, print_function)

try:
    %matplotlib inline
    %config InlineBackend.figure_format='retina'
except:
    pass

# Basic packages
import numpy as np
np.seterr(divide='ignore', invalid='ignore')
import os
import sys
import collections
import glob
import pickle
import pandas

# Packages to work with FITS and (IDL) SME.out files
import astropy.io.fits as pyfits
import astropy.table as table
from astropy.table import Table, hstack, vstack
from scipy.io.idl import readsav

# Matplotlib and associated packages for plotting
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from matplotlib.transforms import Bbox,TransformedBbox
from matplotlib.image import BboxImage
from matplotlib.legend_handler import HandlerBase
from matplotlib._png import read_png
from matplotlib.backends.backend_pdf import PdfPages
from matplotlib.colors import ListedColormap
import matplotlib.colors as colors

params = {
    'font.family'        : 'sans',
    'font.size'          : 17,
    'axes.labelsize'     : 20,
    'ytick.labelsize'    : 16,
    'xtick.labelsize'    : 16,
    'legend.fontsize'    : 20,
    'text.usetex'        : True, 
    'text.latex.preamble': [r'\usepackage{upgreek}', r'\usepackage{amsmath}'],
    }   
plt.rcParams.update(params)

_parula_data = [[0.2081, 0.1663, 0.5292], 
                [0.2116238095, 0.1897809524, 0.5776761905], 
                [0.212252381, 0.2137714286, 0.6269714286], 
                [0.2081, 0.2386, 0.6770857143], 
                [0.1959047619, 0.2644571429, 0.7279], 
                [0.1707285714, 0.2919380952, 0.779247619], 
                [0.1252714286, 0.3242428571, 0.8302714286], 
                [0.0591333333, 0.3598333333, 0.8683333333], 
                [0.0116952381, 0.3875095238, 0.8819571429], 
                [0.0059571429, 0.4086142857, 0.8828428571], 
                [0.0165142857, 0.4266, 0.8786333333], 
                [0.032852381, 0.4430428571, 0.8719571429], 
                [0.0498142857, 0.4585714286, 0.8640571429], 
                [0.0629333333, 0.4736904762, 0.8554380952], 
                [0.0722666667, 0.4886666667, 0.8467], 
                [0.0779428571, 0.5039857143, 0.8383714286], 
                [0.079347619, 0.5200238095, 0.8311809524], 
                [0.0749428571, 0.5375428571, 0.8262714286], 
                [0.0640571429, 0.5569857143, 0.8239571429], 
                [0.0487714286, 0.5772238095, 0.8228285714], 
                [0.0343428571, 0.5965809524, 0.819852381], 
                [0.0265, 0.6137, 0.8135], 
                [0.0238904762, 0.6286619048, 0.8037619048], 
                [0.0230904762, 0.6417857143, 0.7912666667], 
                [0.0227714286, 0.6534857143, 0.7767571429], 
                [0.0266619048, 0.6641952381, 0.7607190476], 
                [0.0383714286, 0.6742714286, 0.743552381], 
                [0.0589714286, 0.6837571429, 0.7253857143], 
                [0.0843, 0.6928333333, 0.7061666667], 
                [0.1132952381, 0.7015, 0.6858571429], 
                [0.1452714286, 0.7097571429, 0.6646285714], 
                [0.1801333333, 0.7176571429, 0.6424333333], 
                [0.2178285714, 0.7250428571, 0.6192619048], 
                [0.2586428571, 0.7317142857, 0.5954285714], 
                [0.3021714286, 0.7376047619, 0.5711857143], 
                [0.3481666667, 0.7424333333, 0.5472666667], 
                [0.3952571429, 0.7459, 0.5244428571], 
                [0.4420095238, 0.7480809524, 0.5033142857], 
                [0.4871238095, 0.7490619048, 0.4839761905], 
                [0.5300285714, 0.7491142857, 0.4661142857], 
                [0.5708571429, 0.7485190476, 0.4493904762],
                [0.609852381, 0.7473142857, 0.4336857143], 
                [0.6473, 0.7456, 0.4188], 
                [0.6834190476, 0.7434761905, 0.4044333333], 
                [0.7184095238, 0.7411333333, 0.3904761905], 
                [0.7524857143, 0.7384, 0.3768142857], 
                [0.7858428571, 0.7355666667, 0.3632714286], 
                [0.8185047619, 0.7327333333, 0.3497904762], 
                [0.8506571429, 0.7299, 0.3360285714], 
                [0.8824333333, 0.7274333333, 0.3217], 
                [0.9139333333, 0.7257857143, 0.3062761905], 
                [0.9449571429, 0.7261142857, 0.2886428571], 
                [0.9738952381, 0.7313952381, 0.266647619], 
                [0.9937714286, 0.7454571429, 0.240347619], 
                [0.9990428571, 0.7653142857, 0.2164142857], 
                [0.9955333333, 0.7860571429, 0.196652381], 
                [0.988, 0.8066, 0.1793666667], 
                [0.9788571429, 0.8271428571, 0.1633142857], 
                [0.9697, 0.8481380952, 0.147452381], 
                [0.9625857143, 0.8705142857, 0.1309], 
                [0.9588714286, 0.8949, 0.1132428571], 
                [0.9598238095, 0.9218333333, 0.0948380952], 
                [0.9661, 0.9514428571, 0.0755333333], 
                [0.9763, 0.9831, 0.0538]]

parula = ListedColormap(_parula_data, name='parula')
parula_zero = _parula_data[0]
parula_0 = ListedColormap(_parula_data, name='parula_0')
parula_0.set_bad((1,1,1))
parula_r = ListedColormap(_parula_data[::-1], name='parula_r')

willi_blau = [0.0722666667, 0.4886666667, 0.8467]

# What shall be combined to a data product?

In [2]:
product_name = 'GBS'

if product_name == 'GBS':
    product_subsets = 'gbs'
    product_input_data_path = '../gbs/data/'
    product_pipeline = 'lbol'

if product_name == 'OpenClusters':
    product_subsets = ['Blanco 1','Pleiades','Hyades','NGC 1817','NGC 1901','ASCC 16','ASCC 20','ASCC 21','NGC 2112','NGC 2204','Berkeley 73','NGC 2232','NGC 2243','Berkeley 33','Berkeley 32','NGC 2516','NGC 2548','NGC 2632','M 67','IC 2602','Melotte 101','Trumpler 20','NGC 5460','NGC 6253','ASCC 89','IC 4665','NGC 6469','NGC 6568','NGC 6583','Ruprecht 145','Ruprecht 147']
    product_input_data_path = '../clusters/data/'
    product_pipeline = 'lbol'

if product_name == 'GlobularClusters':
    product_subsets = ['47 Tuc','NGC 288','NGC 362','NGC 1851','Omega Cen','NGC 6362','NGC 6397','NGC 7099']
    product_input_data_path = '../clusters/data/'
    product_pipeline = 'lbol'

if product_name == 'random10000':
    product_subsets = '10000'
    product_input_data_path = '../random10000/data/'
    product_pipeline = 'lbol'
    
if product_name == 'ts_DR2':
    product_subsets = 'ts_DR2'
    product_input_data_path = '../DR2_rerun/data/'
    product_pipeline = 'lbol'
    
if product_name == 'high_vtot':
    product_subsets = 'high_vtot'
    product_input_data_path = '../science/high_vtot/data/'
    product_pipeline = 'lbol'
    
if product_name == 'Li_rich_giants':
    product_subsets = 'Li_rich'
    product_input_data_path = '../science/Li_rich_giants/data/'
    product_pipeline = 'lbol'
    
if product_name[:4] == '10k_':
    product_subsets = product_name
    product_input_data_path = '../data_products/10k_subsets/'
    product_pipeline = 'lbol'



## Function to gather and combine input data from SME, IRAF, Output structure & A(X)_sun

In [3]:
def get_input_data(product_subsets, product_input_data_path, product_pipeline):
    
    dr3_output_structure = Table.read('../output_structure/galah_dr3_output_structure.fits',1)

    if np.shape(product_subsets)!=():
        joined_sme_data = Table.read(product_input_data_path+'/GALAH_'+product_subsets[0].replace(" ", "")+'_'+product_pipeline+'.fits', format='fits')
        for each in range(1,len(product_subsets)):
            joined_sme_data = vstack([joined_sme_data, Table.read(product_input_data_path+'/GALAH_'+product_subsets[each].replace(" ", "")+'_'+product_pipeline+'.fits', format='fits')])
    else:
        joined_sme_data = Table.read(product_input_data_path+'/GALAH_'+product_subsets.replace(" ", "")+'_'+product_pipeline+'.fits', format='fits')
        
    all_iraf_data = pyfits.getdata('../input/sobject_iraf_53_2MASS_GaiaDR2_WISE_PanSTARRSDR1_BailerJones_K2seis.fits',1)
    iraf_matched = []
    for each in joined_sme_data['SOBJECT_ID']:
        match = np.where(each == all_iraf_data['sobject_id'])[0]
        if len(match)>0:
            iraf_matched.append(match[0])
        else:
            print('No match found for '+str(each))
    iraf_matched = np.array(iraf_matched)
    iraf_data = all_iraf_data[iraf_matched]
    
    abundance_zeropoints= Table.read('../abundance_zeropoints/galahdr3_abundance_zeropoints.fits',1)
    
    return dr3_output_structure, joined_sme_data, iraf_data, abundance_zeropoints

In [4]:
def compute_logg_uncertainty(final_output_data, mc_sampling=10000):
    
    np.random.seed(12)
    
    mc_teff = np.random.normal(
        loc = final_output_data['teff'],
        scale = final_output_data['e_teff'],
        size=(mc_sampling,len(final_output_data['teff']))
    )
    mc_mass = np.random.normal(
        loc = final_output_data['mass'],
        scale = 0.1*final_output_data['mass'],
        size=(mc_sampling,len(final_output_data['mass']))
    )
    mc_mass.clip(min=0., out=mc_mass)
    mc_kmag = np.random.normal(
        loc = final_output_data['ks_m'],
        scale = final_output_data['ks_msigcom'],
        size=(mc_sampling,len(final_output_data['ks_m']))
    )
    mc_bc = np.random.normal(
        loc = final_output_data['bc_ks'],
        scale = final_output_data['e_bc_ks'],
        size=(mc_sampling,len(final_output_data['bc_ks']))
    )
    mc_dist_lo = np.abs(np.random.normal(
        loc = 0,
        scale = final_output_data['r_est'] - final_output_data['r_lo'],
        size=(mc_sampling,len(final_output_data['r_est'])))
    )
    mc_dist_hi = np.abs(np.random.normal(
        loc = 0,
        scale = final_output_data['r_hi'] - final_output_data['r_est'],
        size=(mc_sampling,len(final_output_data['r_est'])))
    )
    select_dist_lo_hi = (np.random.uniform(0, 1, size=(mc_sampling,len(final_output_data['r_est']))) < 0.5)
    mc_dist = np.array(final_output_data['r_est']).T + select_dist_lo_hi*mc_dist_hi - (1-select_dist_lo_hi)*mc_dist_lo
    mc_dist.clip(min=0.001, out=mc_dist)
    e_a_ks = (final_output_data['e_a_ks']).clip(min=0.01)
    mc_ak = np.random.normal(
        loc = final_output_data['a_ks'],
        scale = e_a_ks,
        size=(mc_sampling,len(final_output_data['a_ks']))
    )
    mc_ak.clip(min=0., out=mc_ak)
    
    def logg_function(teff, mass, kmag, bc, dist, ak):
        return(4.438 + 4*np.log10(teff/5772.) + np.log10(mass) + 0.4*(kmag + bc - 5.*np.log10(dist) + 5 - ak - 4.7554))

    mc_logg = logg_function(mc_teff, mc_mass, mc_kmag, mc_bc, mc_dist, mc_ak)

    logg_mean = np.array([np.nanmean(mc_logg[:,x]) for x in range(np.shape(mc_logg)[1])])
    logg_std  = np.array([np.nanstd(mc_logg[:,x]) for x in range(np.shape(mc_logg)[1])])

    return(logg_std)

In [5]:
def combine_SME_IRAF_to_FINAL(output_filename, product_pipeline, sme_data, iraf_data, dr3_output_structure, abundance_zeropoints):
    
    final_output_data = collections.OrderedDict()
    
    abundances_in_mode = np.array([(sme_data['MODE'][0,it]).replace(" ","") for it in range(len(sme_data['MODE'][0]))])
    
    for each_key in dr3_output_structure.keys():
        # Keys in output not matching input keys
        if each_key=='star_id':
            final_output_data[each_key]=np.array(iraf_data['tmass_id'])
        elif each_key in ['sobject_id']:
            final_output_data[each_key]=np.array(sme_data[each_key.upper()])
        elif each_key in ['source_id','ra','ra_error','dec','dec_error','r_est','r_lo','r_hi','red_flag','ebv','snr_c2_iraf','flag_guess','rv_guess','e_rv_guess','teff_guess','logg_guess','feh_guess','j_m','j_msigcom','h_m','h_msigcom','ks_m','ks_msigcom','ph_qual_tmass','w2mpro','w2mpro_error','ph_qual_wise','parallax','parallax_error','astrometric_excess_noise_sig','astrometric_matched_observations','phot_g_mean_mag','bp_rp']:
            final_output_data[each_key]=np.array(iraf_data[each_key])
        elif each_key=='wg4_field':
            if product_pipeline == 'lbol':
                final_output_data[each_key]=np.array([sme_data['FIELD'][x][:-5] for x in range(len(sme_data['FIELD']))])
                final_output_data['wg4_pipeline']=np.array(['lbol' for x in range(len(sme_data['FIELD']))])
            elif product_pipeline == 'seis':
                final_output_data[each_key]=np.array([sme_data['FIELD'][x][:-5] for x in range(len(sme_data['FIELD']))])
                final_output_data['wg4_pipeline']=np.array(['seis' for x in range(len(sme_data['FIELD']))])
            else:
                final_output_data[each_key]=np.array(sme_data['FIELD'])
                final_output_data['wg4_pipeline']=np.array(['free' for x in range(len(sme_data['FIELD']))])
        elif each_key in ['teff', 'e_teff', 'logg', 'e_logg', 'vmic', 'e_vmic', 'mass', 'lbol', 'age', 'alpha_fe', 'e_alpha_fe']:
            final_output_data[each_key]=np.array(sme_data[each_key.upper()])
        elif each_key == 'fe_h_atmo':
            final_output_data[each_key]=np.array(sme_data['FEH'])
        elif each_key == 'e_fe_h_atmo':
            final_output_data[each_key]=np.array(sme_data['E_FEH'])
        elif each_key == 'cov_e_fe_h_atmo':
            final_output_data[each_key]=np.array(sme_data['C_FEH'])
        elif each_key[:6]=='cov_e_':
            if each_key[6:] in ['teff', 'logg']:
                final_output_data[each_key]=np.array(sme_data['C_'+each_key[6:].upper()])
        elif each_key=='fe_h':
            final_output_data['fe_h']=np.array(sme_data['A_ABUND'][:,1] - abundance_zeropoints['A_Fe'][0])
            final_output_data['e_fe_h']=np.array(sme_data['E_ABUND'][:,1])                
        elif each_key=='vbroad':
            final_output_data[each_key]=np.array(sme_data['VSINI'])
            final_output_data['e_'+each_key]=np.array(sme_data['E_VSINI'])
            final_output_data['cov_'+each_key]=np.array(sme_data['C_VSINI'])
        elif each_key=='rv_galah':
            final_output_data[each_key]=np.array(sme_data['VEL'])
            final_output_data['e_'+each_key]=np.array(sme_data['E_VEL'])
            final_output_data['cov_e_'+each_key]=np.array(sme_data['C_VEL'])
        elif each_key=='rv_gaia':
            final_output_data[each_key]=np.array(iraf_data['radial_velocity'])
            final_output_data['e_'+each_key]=np.array(iraf_data['radial_velocity_error'])
        elif each_key=='chi2_sp':
            final_output_data[each_key]=np.array(sme_data['CHI'][:,0])
        elif each_key=='a_ks':

            # Apply RJCE method
            rjce_ak = np.array(0.918*(iraf_data['h_m'] - iraf_data['w2mpro'] - 0.08))
            e_rjce_ak = 0.918*np.sqrt(iraf_data['h_msigcom']**2. + iraf_data['w2mpro_error']**2.)
            
            rjce_ak.clip(min=0.0, out=rjce_ak)
            
            # check if 2MASS Hmag and WISE W2mag have good quality
            tmass_adjusted = np.array(iraf_data['ph_qual_tmass'])
            tmass_adjusted[np.where(tmass_adjusted=='   ')[0]] = 'UUU'
            wise_adjusted = np.array(iraf_data['ph_qual_wise'])
            wise_adjusted[np.where(wise_adjusted=='    ')[0]] = 'UUUU'
            h_m_qual = np.array([tmass_adjusted[x][1] == 'A' for x in range(len(tmass_adjusted))])
            w2_qual = np.array([wise_adjusted[x][1] == 'A' for x in range(len(wise_adjusted))])

            # if photometry bad, exchange a_k by ebv approximation
            bad_rjce = np.isnan(iraf_data['h_m']) | np.isnan(iraf_data['w2mpro']) | (h_m_qual==False) | (w2_qual==False)
            ebv_ak = 0.36*iraf_data['ebv']
            rjce_ak[bad_rjce] = ebv_ak[bad_rjce]
            e_rjce_ak[bad_rjce] = ebv_ak[bad_rjce]
            
            final_output_data['a_ks']=rjce_ak
            final_output_data['e_a_ks']=e_rjce_ak
            
            # adjust too large E(B-V) if photometry good
            bad_ebv = np.where(0.36*iraf_data['ebv'] > 3*rjce_ak)[0]
            if len(bad_ebv) > 0:
                final_output_data['ebv'][bad_ebv] = 2.78*rjce_ak[bad_ebv]
            
            # nearby stars with ebv=0 and a_ks=0
            nearby = np.where(iraf_data['r_est'] <= 100.)[0]
            final_output_data['ebv'][nearby] = 0.
            final_output_data['a_ks'][nearby] = 0.
            final_output_data['e_a_ks'][nearby] = 0.
           
        # Abundance keys
        elif (each_key[0:2]=='A_') & (each_key != 'A_Ks'):
            element = each_key[2:]
            element_in_sme_data = np.where(element == abundances_in_mode)[0]
            if len(element_in_sme_data) != 1:
                # print('Exchange values for '+element)
                if element == 'K': 
                    element_in_sme_data = np.where('K7699' == abundances_in_mode)[0]
                if element == 'Ca': 
                    element_in_sme_data = np.where('Ca5862' == abundances_in_mode)[0]
                if element == 'Cu': 
                    element_in_sme_data = np.where('Cu5782' == abundances_in_mode)[0]
                if element == 'Ba': 
                    element_in_sme_data = np.where('Ba5854' == abundances_in_mode)[0]
            else:
                pass

            element_in_sme_data=element_in_sme_data[0]
            final_output_data[each_key]=np.array(sme_data['A_ABUND'][:,element_in_sme_data])

            final_output_data['e_'+each_key]=np.array(sme_data['E_ABUND'][:,element_in_sme_data])
            final_output_data['flag_'+each_key]=np.array(sme_data['AFLAG'][:,element_in_sme_data])

            final_output_data['flux_'+each_key]=np.array(sme_data['LINEFLUX'][:,element_in_sme_data])
            final_output_data['chi_'+each_key]=np.array(sme_data['CHI'][:,element_in_sme_data])

            if element!='Fe':
                final_output_data[element+'_fe']=np.array(
                    # [X/Fe] = [X/H] - [Fe/H] = A(X) - A(X)_sun - (A(Fe) - A(Fe)_sun)
                    sme_data['A_ABUND'][:,element_in_sme_data]- abundance_zeropoints[each_key][0] 
                    - (final_output_data['A_Fe'] - abundance_zeropoints['A_Fe'][0])
                )
                #final_output_data[element+'_fe']=np.array(sme_data['ABUND'][:,element_in_sme_data])
                final_output_data['e_'+element+'_fe']=np.array(sme_data['E_ABUND'][:,element_in_sme_data])
        # Already filled
        elif ((each_key in ['wg4_pipeline', 'e_fe_h', 'e_rv_galah', 'c_rv_galah', 'e_rv_gaia', 'e_vbroad', 'e_a_ks']) | (each_key[:4]=='e_A_') | (each_key[:7]=='flag_A_') | (each_key[:7]=='flux_A_') | (each_key[:7]=='chi2_A_') | (each_key[-3:]=='_fe')):
            pass
        # Placeholder
        elif each_key in ['wg4_sp_flag', 'e_mass', 'e_lbol', 'e_age', 'bc_ks', 'e_bc_ks']:
            final_output_data[each_key] = np.array([np.NaN for x in range(len(sme_data['FIELD']))])
        else:
            print('No match for '+each_key)
    
    def bc_function(lbol, ks_m, dist, ak):
        return(-2.5*np.log10(lbol) - ks_m - 5.*np.log10(1./dist) - 5. + 4.7554)

    # Reverse engineer BC for Ks based on bolometric luminosity
    final_output_data['bc_ks'] = bc_function(
        final_output_data['lbol'],
        final_output_data['ks_m'],
        final_output_data['r_est'],
        final_output_data['a_ks']
    )

    # Assume the error on BC for Ks is 0.1 mag
    final_output_data['e_bc_ks'] = 0.1 * np.ones(len(final_output_data['bc_ks']))

    # MC sample the uncertainties for logg
    final_output_data['e_logg'] = compute_logg_uncertainty(final_output_data)
            
    output_pandas = pandas.DataFrame(final_output_data,columns=final_output_data.keys())
    output_astropy = Table.from_pandas(output_pandas)
    output_astropy.write('../data_products/GALAH_iDR3_'+output_filename+'.fits',overwrite=True)

    return output_astropy

## Execute

In [6]:
dr3_output_structure, sme_data, iraf_data, abundance_zeropoints = get_input_data(
    product_subsets,
    product_input_data_path,
    product_pipeline)

output_astropy = combine_SME_IRAF_to_FINAL(
    output_filename = product_name, 
    product_pipeline=product_pipeline,
    sme_data=sme_data,
    iraf_data=iraf_data,
    dr3_output_structure=dr3_output_structure,
    abundance_zeropoints=abundance_zeropoints);

output_astropy

/Users/buder/anaconda/lib/python2.7/site-packages/numpy/lib/nanfunctions.py:703: RuntimeWarning: Mean of empty slice
  warnings.warn("Mean of empty slice", RuntimeWarning)
/Users/buder/anaconda/lib/python2.7/site-packages/numpy/lib/nanfunctions.py:1202: RuntimeWarning: Degrees of freedom <= 0 for slice.
  warnings.warn("Degrees of freedom <= 0 for slice.", RuntimeWarning)


star_id,sobject_id,source_id,wg4_field,wg4_pipeline,wg4_sp_flag,teff,e_teff,cov_e_teff,logg,e_logg,cov_e_logg,fe_h,e_fe_h,fe_h_atmo,e_fe_h_atmo,cov_e_fe_h_atmo,vmic,e_vmic,vbroad,e_vbroad,cov_vbroad,mass,e_mass,lbol,e_lbol,age,e_age,alpha_fe,e_alpha_fe,chi2_sp,A_Fe,e_A_Fe,flag_A_Fe,flux_A_Fe,chi_A_Fe,A_Li,e_A_Li,flag_A_Li,flux_A_Li,chi_A_Li,Li_fe,e_Li_fe,A_C,e_A_C,flag_A_C,flux_A_C,chi_A_C,C_fe,e_C_fe,A_O,e_A_O,flag_A_O,flux_A_O,chi_A_O,O_fe,e_O_fe,A_O7772,e_A_O7772,flag_A_O7772,flux_A_O7772,chi_A_O7772,O7772_fe,e_O7772_fe,A_O7774,e_A_O7774,flag_A_O7774,flux_A_O7774,chi_A_O7774,O7774_fe,e_O7774_fe,A_O7775,e_A_O7775,flag_A_O7775,flux_A_O7775,chi_A_O7775,O7775_fe,e_O7775_fe,A_Na,e_A_Na,flag_A_Na,flux_A_Na,chi_A_Na,Na_fe,e_Na_fe,A_Na4752,e_A_Na4752,flag_A_Na4752,flux_A_Na4752,chi_A_Na4752,Na4752_fe,e_Na4752_fe,A_Na5683,e_A_Na5683,flag_A_Na5683,flux_A_Na5683,chi_A_Na5683,Na5683_fe,e_Na5683_fe,A_Na5688,e_A_Na5688,flag_A_Na5688,flux_A_Na5688,chi_A_Na5688,Na5688_fe,e_Na5688_fe,A_Mg,e_A_Mg,flag_A_Mg,flux_A_Mg,chi_A_Mg,Mg_fe,e_Mg_fe,A_Mg4730,e_A_Mg4730,flag_A_Mg4730,flux_A_Mg4730,chi_A_Mg4730,Mg4730_fe,e_Mg4730_fe,A_Mg5711,e_A_Mg5711,flag_A_Mg5711,flux_A_Mg5711,chi_A_Mg5711,Mg5711_fe,e_Mg5711_fe,A_Mg7692,e_A_Mg7692,flag_A_Mg7692,flux_A_Mg7692,chi_A_Mg7692,Mg7692_fe,e_Mg7692_fe,A_Al,e_A_Al,flag_A_Al,flux_A_Al,chi_A_Al,Al_fe,e_Al_fe,A_Al6696,e_A_Al6696,flag_A_Al6696,flux_A_Al6696,chi_A_Al6696,Al6696_fe,e_Al6696_fe,A_Al6699,e_A_Al6699,flag_A_Al6699,flux_A_Al6699,chi_A_Al6699,Al6699_fe,e_Al6699_fe,A_Al7835,e_A_Al7835,flag_A_Al7835,flux_A_Al7835,chi_A_Al7835,Al7835_fe,e_Al7835_fe,A_Al7836,e_A_Al7836,flag_A_Al7836,flux_A_Al7836,chi_A_Al7836,Al7836_fe,e_Al7836_fe,A_Si,e_A_Si,flag_A_Si,flux_A_Si,chi_A_Si,Si_fe,e_Si_fe,A_Si5666,e_A_Si5666,flag_A_Si5666,flux_A_Si5666,chi_A_Si5666,Si5666_fe,e_Si5666_fe,A_Si5684,e_A_Si5684,flag_A_Si5684,flux_A_Si5684,chi_A_Si5684,Si5684_fe,e_Si5684_fe,A_Si5690,e_A_Si5690,flag_A_Si5690,flux_A_Si5690,chi_A_Si5690,Si5690_fe,e_Si5690_fe,A_Si5701,e_A_Si5701,flag_A_Si5701,flux_A_Si5701,chi_A_Si5701,Si5701_fe,e_Si5701_fe,A_Si5772,e_A_Si5772,flag_A_Si5772,flux_A_Si5772,chi_A_Si5772,Si5772_fe,e_Si5772_fe,A_Si5793,e_A_Si5793,flag_A_Si5793,flux_A_Si5793,chi_A_Si5793,Si5793_fe,e_Si5793_fe,A_Si6722,e_A_Si6722,flag_A_Si6722,flux_A_Si6722,chi_A_Si6722,Si6722_fe,e_Si6722_fe,A_K5802,e_A_K5802,flag_A_K5802,flux_A_K5802,chi_A_K5802,K5802_fe,e_K5802_fe,A_K7699,e_A_K7699,flag_A_K7699,flux_A_K7699,chi_A_K7699,K7699_fe,e_K7699_fe,A_K,e_A_K,flag_A_K,flux_A_K,chi_A_K,K_fe,e_K_fe,A_Ca5862,e_A_Ca5862,flag_A_Ca5862,flux_A_Ca5862,chi_A_Ca5862,Ca5862_fe,e_Ca5862_fe,A_Ca,e_A_Ca,flag_A_Ca,flux_A_Ca,chi_A_Ca,Ca_fe,e_Ca_fe,A_Ca6501,e_A_Ca6501,flag_A_Ca6501,flux_A_Ca6501,chi_A_Ca6501,Ca6501_fe,e_Ca6501_fe,A_Ca5857,e_A_Ca5857,flag_A_Ca5857,flux_A_Ca5857,chi_A_Ca5857,Ca5857_fe,e_Ca5857_fe,A_Ca5868,e_A_Ca5868,flag_A_Ca5868,flux_A_Ca5868,chi_A_Ca5868,Ca5868_fe,e_Ca5868_fe,A_Ca6494,e_A_Ca6494,flag_A_Ca6494,flux_A_Ca6494,chi_A_Ca6494,Ca6494_fe,e_Ca6494_fe,A_Ca6500,e_A_Ca6500,flag_A_Ca6500,flux_A_Ca6500,chi_A_Ca6500,Ca6500_fe,e_Ca6500_fe,A_Ca6509,e_A_Ca6509,flag_A_Ca6509,flux_A_Ca6509,chi_A_Ca6509,Ca6509_fe,e_Ca6509_fe,A_Sc,e_A_Sc,flag_A_Sc,flux_A_Sc,chi_A_Sc,Sc_fe,e_Sc_fe,A_Ti,e_A_Ti,flag_A_Ti,flux_A_Ti,chi_A_Ti,Ti_fe,e_Ti_fe,A_V,e_A_V,flag_A_V,flux_A_V,chi_A_V,V_fe,e_V_fe,A_Cr,e_A_Cr,flag_A_Cr,flux_A_Cr,chi_A_Cr,Cr_fe,e_Cr_fe,A_Mn,e_A_Mn,flag_A_Mn,flux_A_Mn,chi_A_Mn,Mn_fe,e_Mn_fe,A_Co,e_A_Co,flag_A_Co,flux_A_Co,chi_A_Co,Co_fe,e_Co_fe,A_Co7713,e_A_Co7713,flag_A_Co7713,flux_A_Co7713,chi_A_Co7713,Co7713_fe,e_Co7713_fe,A_Ni,e_A_Ni,flag_A_Ni,flux_A_Ni,chi_A_Ni,Ni_fe,e_Ni_fe,A_Cu5700,e_A_Cu5700,flag_A_Cu5700,flux_A_Cu5700,chi_A_Cu5700,Cu5700_fe,e_Cu5700_fe,A_Cu5782,e_A_Cu5782,flag_A_Cu5782,flux_A_Cu5782,chi_A_Cu5782,Cu5782_fe,e_Cu5782_fe,A_Cu,e_A_Cu,flag_A_Cu,flux_A_Cu,chi_A_Cu,Cu_fe,e_Cu_fe,A_Zn,e_A_Zn,flag_A_Zn,flux_A_Zn,chi_A_Zn,Zn_fe,e_Zn_fe,A_Zn4722,e_A_Zn4722,flag_A_Zn4722,flux_A_Zn4722,chi_A_Zn4722,Zn4722_fe,e_Zn4722_fe,A_Zn4811,e_A_Zn4811,flag_A_Zn481